In [25]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from scipy import stats
from statsmodels.stats.multicomp import MultiComparison

In [26]:
# Cargando el dataframe
file_path = './data/DataImputed.csv'
df = pd.read_csv(file_path)

# Mostrando las primeras filas para entender la estructura y los tipos de datos
df.head()

,Unnamed: 0,Fecha,Linea,Horas,Qty_passangers,festivos
0,0,2021-02-18,1,5,5807.0,0
1,1,2021-03-01,1,5,5982.0,0
2,2,2021-06-12,1,5,4493.0,0
3,3,2021-06-20,1,5,1296.0,0
4,4,2021-07-15,1,5,6227.0,0


## Test de anova para detectar diferencias significativas entre los días de la semana para cada combinación única de línea y hora


In [ ]:
import pandas as pd
from scipy.stats import f_oneway
import seaborn as sns
import matplotlib.pyplot as plt

def realizar_anova(archivo_csv):
    """
    Realiza un análisis de varianza (ANOVA) para determinar si hay diferencias significativas
    en la cantidad de pasajeros entre los días de la semana para cada combinación única de línea y hora.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV que contiene los datos.

    Retorna:
    - df_resultados (pd.DataFrame): DataFrame que contiene los resultados de las pruebas de ANOVA.

    Explicación:
    Un valor p menor a 0.05 indica que hay evidencia suficiente para rechazar la hipótesis nula.
    En este contexto, la hipótesis nula (H0) asume que las medias de la cantidad de pasajeros son iguales
    para todos los días de la semana en una línea y hora específicas. La hipótesis alternativa (H1)
    sugiere que al menos una de las medias es diferente.

    Por lo tanto:
    - Si Valor_p < 0.05: Se rechaza H0, lo que sugiere que hay diferencias significativas en la cantidad
      de pasajeros entre los días de la semana para esa combinación específica de línea y hora.
    - Si Valor_p >= 0.05: No hay evidencia suficiente para rechazar H0, lo que indica que no hay
      diferencias significativas en la cantidad de pasajeros entre los días de la semana.

    """

    # Lectura del DataFrame desde un archivo CSV y manipulación de datos
    df = pd.read_csv(archivo_csv).drop(['Unnamed: 0'], axis=1)
    
    # Manipulación de fechas
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['dia_semana'] = df['Fecha'].dt.day_name()

    # Inicialización de la lista de resultados de ANOVA
    resultados_anova = []

    # Bucle para realizar pruebas de ANOVA para cada combinación de línea y hora
    for linea in df['Linea'].unique():
        for hora in df['Horas'].unique():
            # Filtrar el DataFrame por línea y hora
            subset = df[(df['Linea'] == linea) & (df['Horas'] == hora)]

            # Crear grupos para la prueba de ANOVA
            grupos = [subset[subset['Fecha'].dt.day_name() == dia]['Qty_passangers'] for dia in df['dia_semana'].unique()]

            # Realizar la prueba de ANOVA
            resultado_anova = f_oneway(*grupos)

            # Almacenar los resultados en la lista
            resultados_anova.append({
                'Linea': linea,
                'Hora': hora,
                'Estadistica': resultado_anova.statistic,
                'Valor_p': resultado_anova.pvalue
            })

    # Convertir la lista de resultados en un DataFrame
    df_resultados = pd.DataFrame(resultados_anova)

    return df_resultados[df_resultados['Valor_p'] > 0.05]

# Uso de la función
resultado_anova = realizar_anova('./data/DataImputed.csv')
resultado_anova.sort_values(by=['Valor_p'], ascending=True)

,Linea,Hora,Estadistica,Valor_p
101,J,16,1.452363,0.191701
130,M,9,1.028051,0.405410
94,J,9,0.935044,0.468833


# KRUSKAL

In [3]:
import pandas as pd
from scipy.stats import kruskal

def realizar_kruskal_wallis(archivo_csv):
    """
    Realiza un test de Kruskal-Wallis para determinar si hay diferencias significativas
    en la cantidad de pasajeros entre los días de la semana para cada combinación única de línea y hora.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV que contiene los datos.

    Retorna:
    - df_resultados (pd.DataFrame): DataFrame que contiene los resultados del test de Kruskal-Wallis.

    Explicación:
    Un valor p menor a 0.05 indica que hay evidencia suficiente para rechazar la hipótesis nula.
    La hipótesis nula (H0) asume que no hay diferencias significativas en la distribución de la cantidad
    de pasajeros entre los días de la semana para una línea y hora específicas.

    Por lo tanto:
    - Si Valor_p < 0.05: Se rechaza H0, lo que sugiere que hay diferencias significativas.
    - Si Valor_p >= 0.05: No hay evidencia suficiente para rechazar H0.
    """

    # Lectura del DataFrame desde un archivo CSV y manipulación de datos
    df = pd.read_csv(archivo_csv).drop(['Unnamed: 0'], axis=1)
    
    # Manipulación de fechas
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['dia_semana'] = df['Fecha'].dt.day_name()

    # Inicialización de la lista de resultados de Kruskal-Wallis
    resultados_kruskal_wallis = []

    # Bucle para realizar pruebas de Kruskal-Wallis para cada combinación de línea y hora
    for linea in df['Linea'].unique():
        for hora in df['Horas'].unique():
            # Filtrar el DataFrame por línea y hora
            subset = df[(df['Linea'] == linea) & (df['Horas'] == hora)]

            # Crear grupos para la prueba de Kruskal-Wallis
            grupos = [subset[subset['Fecha'].dt.day_name() == dia]['Qty_passangers'] for dia in df['dia_semana'].unique()]

            # Realizar la prueba de Kruskal-Wallis
            resultado_kruskal_wallis = kruskal(*grupos)

            # Almacenar los resultados en la lista
            resultados_kruskal_wallis.append({
                'Linea': linea,
                'Hora': hora,
                'Estadistica': resultado_kruskal_wallis.statistic,
                'Valor_p': resultado_kruskal_wallis.pvalue
            })

    # Convertir la lista de resultados en un DataFrame
    df_resultados = pd.DataFrame(resultados_kruskal_wallis)

    return df_resultados[df_resultados['Valor_p'] > 0.05]

# Uso de la función
resultado_kruskal_wallis = realizar_kruskal_wallis('./data/DataImputed.csv')
resultado_kruskal_wallis.sort_values(by=['Valor_p'], ascending=True)


,Linea,Hora,Estadistica,Valor_p
94,J,9,11.759206,0.067561


## Test de anova Por Día de la Semana: Analizar si la afluencia varía significativamente entre los diferentes días de la semana.

Por Hora del Día: Investigar si hay diferencias significativas en la cantidad de pasajeros en diferentes horas del día.

Por Días Festivos vs. Días No Festivos: Comparar la afluencia en días festivos frente a días no festivos para ver si hay una diferencia significativa.

In [66]:
import pandas as pd
from scipy.stats import f_oneway

def asignar_periodo_del_dia(hora, periodos):
    """
    Asigna un período del día a una hora dada, basado en los rangos definidos en 'periodos'.
    """
    for periodo, (inicio, fin) in periodos.items():
        if inicio <= hora < fin:
            return periodo
    return 'Otro'  # para horas fuera de los rangos especificados

def realizar_anova_por_periodos(archivo_csv, periodos):
    """
    Realiza un análisis ANOVA para cada línea dentro de cada período del día y retorna tres DataFrames separados.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV.
    - periodos (dict): Diccionario que define los períodos del día.
    """
    # Leer los datos
    df = pd.read_csv(archivo_csv)
    
    # Convertir la fecha a datetime y crear la columna de período del día
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Periodo_Dia'] = df['Horas'].apply(lambda hora: asignar_periodo_del_dia(hora, periodos))
    
    # Diccionario para almacenar los DataFrames de resultados
    resultados_por_periodo = {periodo: [] for periodo in periodos}

    # Realizar ANOVA para cada línea y período
    for periodo in periodos:
        for linea in df['Linea'].unique():
            subset = df[(df['Linea'] == linea) & (df['Periodo_Dia'] == periodo)]
            grupos = [grupo['Qty_passangers'].values for _, grupo in subset.groupby(subset['Fecha'].dt.date) if len(grupo) > 1]

            if grupos and len(grupos) > 1:
                resultado_anova = f_oneway(*grupos)
                resultados_por_periodo[periodo].append({
                    'Linea': linea,
                    'Estadistica_F': resultado_anova.statistic,
                    'Valor_p': resultado_anova.pvalue
                })

    # Convertir los resultados a DataFrames
    for periodo in resultados_por_periodo:
        resultados_por_periodo[periodo] = pd.DataFrame(resultados_por_periodo[periodo])

    return resultados_por_periodo['Mañana'], resultados_por_periodo['Tarde'], resultados_por_periodo['Noche']

# Ejemplo de uso de la función
periodos_definidos = {'Mañana': (6, 12), 'Tarde': (12, 18), 'Noche': (18, 23)}
resultados_manana, resultados_tarde, resultados_noche = realizar_anova_por_periodos('./data/DataImputed.csv', periodos_definidos)


In [67]:
resultados_manana_significativos = resultados_manana[resultados_manana['Valor_p'] > 0.05]
resultados_tarde_significativos = resultados_tarde[resultados_tarde['Valor_p'] > 0.05]
resultados_noche_significativos = resultados_noche[resultados_noche['Valor_p'] > 0.05]

# Mostrar los resultados
print("Resultados Mañana (p > 0.05):")
print(resultados_manana_significativos)
print("\nResultados Tarde (p > 0.05):")
print(resultados_tarde_significativos)
print("\nResultados Noche (p > 0.05):")
print(resultados_noche_significativos)

Resultados Mañana (p > 0.05):
  Linea  Estadistica_F   Valor_p
4     H       0.734499  1.000000
5     J       0.886215  0.989021
6     K       1.050874  0.165700
7     M       0.868843  0.996274

Resultados Tarde (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_F, Valor_p]
Index: []

Resultados Noche (p > 0.05):
Empty DataFrame
Columns: [Linea, Estadistica_F, Valor_p]
Index: []


Resultados
Valor F: 281.21
Valor p: Prácticamente 0 (indicativo de una probabilidad extremadamente baja de que las diferencias observadas en las medias sean debido al azar).

El alto valor F y el valor p cercano a cero indican que hay diferencias significativas en la afluencia de pasajeros entre los distintos días de la semana. Esto sugiere que el día de la semana es un factor importante que influye en la cantidad de pasajeros que utilizan el metro. La significancia estadística implica que estas diferencias no son atribuibles al azar y que hay patrones específicos en la afluencia de pasajeros según el día de la semana.

# KRUSKAL

In [5]:
import pandas as pd
from scipy.stats import kruskal

def asignar_periodo_del_dia(hora, periodos):
    """
    Asigna un período del día a una hora dada, basado en los rangos definidos en 'periodos'.
    """
    for periodo, (inicio, fin) in periodos.items():
        if inicio <= hora < fin:
            return periodo
    return 'Otro'  # para horas fuera de los rangos especificados

def realizar_kruskal_wallis_por_periodos(archivo_csv, periodos):
    """
    Realiza un test de Kruskal-Wallis para cada línea dentro de cada período del día y retorna tres DataFrames separados.

    Parámetros:
    - archivo_csv (str): Ruta al archivo CSV.
    - periodos (dict): Diccionario que define los períodos del día.
    """
    # Leer los datos
    df = pd.read_csv(archivo_csv)
    
    # Convertir la fecha a datetime y crear la columna de período del día
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df['Periodo_Dia'] = df['Horas'].apply(lambda hora: asignar_periodo_del_dia(hora, periodos))
    
    # Diccionario para almacenar los DataFrames de resultados
    resultados_por_periodo = {periodo: [] for periodo in periodos}

    # Realizar Kruskal-Wallis para cada línea y período
    for periodo in periodos:
        for linea in df['Linea'].unique():
            subset = df[(df['Linea'] == linea) & (df['Periodo_Dia'] == periodo)]
            grupos = [grupo['Qty_passangers'].values for _, grupo in subset.groupby(subset['Fecha'].dt.date) if len(grupo) > 1]

            if grupos and len(grupos) > 1:
                resultado_kruskal_wallis = kruskal(*grupos)
                resultados_por_periodo[periodo].append({
                    'Linea': linea,
                    'Estadistica_H': resultado_kruskal_wallis.statistic,
                    'Valor_p': resultado_kruskal_wallis.pvalue
                })

    # Convertir los resultados a DataFrames
    for periodo in resultados_por_periodo:
        resultados_por_periodo[periodo] = pd.DataFrame(resultados_por_periodo[periodo])

    return resultados_por_periodo['Mañana'], resultados_por_periodo['Tarde'], resultados_por_periodo['Noche']

# Ejemplo de uso:
periodos = {'Mañana': (6, 12), 'Tarde': (12, 18), 'Noche': (18, 24)}
manana_df, tarde_df, noche_df = realizar_kruskal_wallis_por_periodos('./data/DataImputed.csv', periodos)


In [8]:
resultados_manana_significativos1 = manana_df[manana_df['Valor_p'] < 0.05]
resultados_tarde_significativos1 = tarde_df[tarde_df['Valor_p'] < 0.05]
resultados_noche_significativos1 = noche_df[noche_df['Valor_p'] < 0.05]

# Mostrar los resultados
print("Resultados Mañana (p > 0.05):")
print(resultados_manana_significativos1)
print("\nResultados Tarde (p > 0.05):")
print(resultados_tarde_significativos1)
print("\nResultados Noche (p > 0.05):")
print(resultados_noche_significativos1)

Resultados Mañana (p > 0.05):
  Linea  Estadistica_H        Valor_p
0     1    3398.399861  7.101658e-283
1     2    4885.538163   0.000000e+00
2     A    3409.595151  1.170254e-284
3     B    3387.642006  3.653902e-281
4     H    1093.545789   8.202894e-06
5     J    1033.701233   9.187515e-04
6     K    1132.732832   4.128960e-08
7     M    1203.224275   9.094648e-11
8     O    4044.028689   0.000000e+00
9   T-A    3616.765582   0.000000e+00

Resultados Tarde (p > 0.05):
  Linea  Estadistica_H        Valor_p
0     1    3591.554839   0.000000e+00
1     2    3934.252241   0.000000e+00
2     A    3119.556644  3.430073e-239
3     B    3293.471908  2.867642e-266
4     H    2745.528823  8.708444e-186
5     J    2883.802380  5.167070e-207
6     K    3117.924237  6.851283e-245
7     M    2046.357374   4.612377e-90
8     O    4279.597434   0.000000e+00
9   T-A    3598.560997   0.000000e+00

Resultados Noche (p > 0.05):
  Linea  Estadistica_H        Valor_p
0     1    1400.787300   1.575612e-2

In [ ]:
# Modelo ANOVA de dos vías para analizar efectos principales y de interacción

modelo = ols('Qty_passangers ~ C(Linea) + C(dia_semana) + C(Linea):C(dia_semana)', data=df).fit()

anova_table = sm.stats.anova_lm(modelo, typ=2)



,sum_sq,df,F,PR(>F)
C(Linea),1.047465e+13,9.0,48903.112956,0.0
C(dia_semana),1.547793e+11,6.0,1083.930297,0.0
C(Linea):C(dia_semana),6.272908e+11,54.0,488.106726,0.0
Residual,3.877348e+12,162920.0,NaN,NaN
